# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI
import numpy as np

import re

import math

from langchain_openai import ChatOpenAI

# to read pdf files
from langchain_community.document_loaders import PyPDFLoader

# to read text of pdf files
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


from langchain import PromptTemplate, LLMChain
from langchain.callbacks import get_openai_callback


# get external variables or functions:
import src.API_key as key

# Load data to feed ChatGPT

In [2]:
## set working environment
#> Get the current working directory
print(os.getcwd())
directory = os.getcwd()

# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

c:\DATEN\PHD\Article_SoftRobotIntervention\Analyses\main study - ChatGPT
['.venv', 'data', 'output', 'runChatGPT.ipynb', 'src', 'v01', 'v02']


## Load Scenario Texts
This should be the final scenario texts in English of the two robots:

* rescue robot
* socially assistive robot


source: https://python.langchain.com/v0.2/docs/tutorials/pdf_qa/

In [3]:
# Load the scenario texts of the rescue robot
file_path = directory + "/data/scenario texts/" + "rescue robot" + ".pdf"
loader = PyPDFLoader(file_path)
doc_RR = loader.load()
print(len(doc_RR))

# Load the scenario texts of the socially assistive robot
file_path = directory + "/data/scenario texts/" + "socially assistive robot" + ".pdf"
loader = PyPDFLoader(file_path)
doc_SAR = loader.load()
print(len(doc_SAR))

6
6


 Using a text splitter, the loaded documents will be split into smaller documents that can more easily fit into an LLM's context window, then load them into a vector store. Then a retriever from the vector store is created for use in our RAG chain.

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits_RR = text_splitter.split_documents(doc_RR)
vectorstore_RR = Chroma.from_documents(documents=splits_RR, embedding=OpenAIEmbeddings(openai_api_key=key.openai_api_key))

retriever_RR = vectorstore_RR.as_retriever()

In [5]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key.openai_api_key)

In [6]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever_RR, question_answer_chain)

results = rag_chain.invoke({"input": "What are the advantages of rescue robots?"})

results

{'input': 'What are the advantages of rescue robots?',
 'context': [Document(metadata={'page': 0, 'source': 'c:\\DATEN\\PHD\\Article_SoftRobotIntervention\\Analyses\\main study - ChatGPT/data/scenario texts/rescue robot.pdf'}, page_content='essential supplies, and autonomously assisting in the rescue of victims, these robots \ncan enhance the efficiency and effectiveness of rescue operations.  \nWhile robots f or search and rescue are still in the development phase, it is important \nto consider the ethical aspects (risks and benefits) of these technologies.'),
  Document(metadata={'page': 0, 'source': 'c:\\DATEN\\PHD\\Article_SoftRobotIntervention\\Analyses\\main study - ChatGPT/data/scenario texts/rescue robot.pdf'}, page_content='––––––––––––––––––  First Page in Experiment ––––––––––––––––––   \nPlease read the following text carefully.  \nAfterwards we will ask you to draw a CAM around the predefined \nknotpoint:  \n"Which risks and benefits come to your mind when considering the 

In [7]:
print(results["context"][0].page_content)
print(results["context"][0].metadata)

essential supplies, and autonomously assisting in the rescue of victims, these robots 
can enhance the efficiency and effectiveness of rescue operations.  
While robots f or search and rescue are still in the development phase, it is important 
to consider the ethical aspects (risks and benefits) of these technologies.
{'page': 0, 'source': 'c:\\DATEN\\PHD\\Article_SoftRobotIntervention\\Analyses\\main study - ChatGPT/data/scenario texts/rescue robot.pdf'}


## Load .xlsx files (lists of words)
This should be the final scenario texts in English of the two robots combined and seperated:

* rescue robot_multipleSheets
* socially assistive robot_multipleSheets
* rescue robot_socially assistive robot_multipleSheets

sources:
* https://python.langchain.com/v0.2/docs/tutorials/llm_chain/ (Build a Simple LLM Application with LCEL)
* https://python.langchain.com/v0.2/docs/how_to/structured_output/ (How to return structured data from a model)
* https://python.langchain.com/v0.2/docs/how_to/llm_token_usage_tracking/ (How to track token usage for LLMs)


In [8]:
## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "rescue robot_socially assistive robot_multipleSheets" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names combined:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_Combined = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}


## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "rescue robot_multipleSheets" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names RR:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_RR = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}



## Load the xlsx file of the rescue robot and the socially assistive robot combined
# Path to your Excel file
file_path = directory + "/data/" + "socially assistive robot_multipleSheets" + ".xlsx"
# Load the Excel file
excel_data = pd.ExcelFile(file_path)
# Print the sheet names
print("Sheet names SAR:", excel_data.sheet_names)
# Load all sheets into a dictionary of dataframes
all_sheets_SAR = {sheet_name: excel_data.parse(sheet_name) for sheet_name in excel_data.sheet_names}


Sheet names combined: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']
Sheet names RR: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']
Sheet names SAR: ['RCPP', 'LC', 'T', 'SIP', 'HRIP', 'AN', 'SIN', 'R', 'HC', 'RCN', 'SA', 'TP', 'TL', 'RCPN', 'HRIN', 'MT', 'RCA', 'AP']


In [9]:
abbreviations_dict = {
    'RCPP': 'perceived positive usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)',
    'LC': 'perceived low costs',
    'T': 'perceived trust',
    'SIP': 'perceived positive social impact',
    'HRIP': 'perceived positive Human-Robot-Interaction',
    'AN': 'perceived negative anthropomorphism',
    'SIN': 'perceived positive social impact',
    'R': 'perceived risks',
    'HC': 'perceived high costs',
    'RCN': 'neutral rest category (rest category refers to a classification of arguments that do not fit into any of the predefined categories)',
    'SA': 'perceived safety',
    'TP': 'perceived technological possibilities',
    'TL': 'perceived technological limitations',
    'RCPN': 'perceived negative usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)',
    'HRIN': 'perceived negative Human-Robot-Interaction',
    'MT': 'perceived mistrust',
    'RCA': 'ambivalent rest category (rest category refers to a classification of arguments that do not fit into any of the predefined categories)',
    'AP': 'perceived positive anthropomorphism'
}

print(abbreviations_dict)
print(abbreviations_dict.keys())
print(abbreviations_dict['AN'])

{'RCPP': 'perceived positive usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)', 'LC': 'perceived low costs', 'T': 'perceived trust', 'SIP': 'perceived positive social impact', 'HRIP': 'perceived positive Human-Robot-Interaction', 'AN': 'perceived negative anthropomorphism', 'SIN': 'perceived positive social impact', 'R': 'perceived risks', 'HC': 'perceived high costs', 'RCN': 'neutral rest category (rest category refers to a classification of arguments that do not fit into any of the predefined categories)', 'SA': 'perceived safety', 'TP': 'perceived technological possibilities', 'TL': 'perceived technological limitations', 'RCPN': 'perceived negative usefulness (rest category, refers to a classification of arguments that do not fit into any of the predefined categories)', 'HRIN': 'perceived negative Human-Robot-Interaction', 'MT': 'perceived mistrust', 'RCA': 'ambivalent rest category (rest category refers to a c

In [10]:
# Example: Access data from a specific sheet
sheet_name = 'T'
print(all_sheets_Combined[sheet_name].shape)
print(all_sheets_RR[sheet_name].shape)
print(all_sheets_SAR[sheet_name].shape)

(88, 6)
(41, 6)
(47, 6)


Use dictionaries to map each word to its comment.

In [11]:
def create_multivalue_dict(df, key_col, value_col):
    """
    Create a dictionary from a DataFrame where each key maps to a list of values.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    key_col (str): The column name to be used as keys.
    value_col (str): The column name to be used as values.
    
    Returns:
    dict: A dictionary where each key maps to a list of values.
    """
    # Remove rows with NaN in the key columns
    df = df.dropna(subset=[key_col])

    # Create a dictionary to map items to their comments, allowing for multiple comments per key
    multivalue_dict = {}
    for key, value in zip(df[key_col], df[value_col]):
        if key in multivalue_dict:
            multivalue_dict[key].append(value)
        else:
            multivalue_dict[key] = [value]

    return multivalue_dict

# Example usage
data = {
    'constant': ['a', 'b', 'a', 'c', 'b', np.nan],
    'constant_comments': ['comment1', 'comment2', 'comment3', 'comment4', 'comment5', 'comment6']
}
df = pd.DataFrame(data)

df_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print(df_mapping)

{'a': ['comment1', 'comment3'], 'b': ['comment2', 'comment5'], 'c': ['comment4']}


In [12]:
# Remove rows with NaN in the key columns as they cannot be used as dictionary keys
#> not sensitive to multiple identical keys: dict(zip(df['constant'], df['constant_comments']))
df = all_sheets_Combined[sheet_name]

constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
print("mapping constant x comments:", constant_comments_mapping)
print(len(constant_comments_mapping))

new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
print("mapping new x comments:", new_comments_mapping)
print(len(new_comments_mapping))

mapping constant x comments: {'constant availability (timewise)': [nan], 'unconditional emotional accompaniment': [nan], 'discreet': ['Stop complaining'], 'Less abuse weak': ['There is unspeakable abuse of vulnerable individuals by nursing staff. I read a newspaper article where a person had installed a hidden camera in the room of their elderly mother (to find out who was stealing), and it turned out there was severe, severe abuse by nursing staff.'], 'People think': [nan], 'exact task repetition': [nan], 'robot not sick': [nan], 'neutral': ['Social assistance robots are neutral and do not swing from emotional state to positive or negative'], 'objective': ["No condemnation or falsification of feelings. Soberly considers the facts. // The robot doesn't care who you are."], 'fact-based': ['Assistance robots proceed schematically according to facts and can refer to scientific data free from emotions. // Scientific elaboration and programming of behaviors can achieve a desired reaction. S

# Set up ChatGPT

## Provide prompt template

In [13]:
system_template = """<Context>You are a researcher tasked with summarizing two wordlists that highlight people's assessments of rigid robots compared to soft robots. Laypersons were informed about the potential risks and benefits of {robots} through scenario texts. Initially, they listed their perceived risks and benefits of rigid robots in a list titled "rigid." Subsequently, they learned about the trend towards soft robots, which are made of flexible, soft materials and are electronic-free. They then created a list titled "soft" to highlight the differences between rigid and soft robots. The overarching topic of the two lists is the {topicCategory}.</Context>

<Data Structure>The lists "rigid" and "soft" are dictionaries where the keys are written arguments, and their corresponding values are one or more comments related to those arguments. The value [nan] indicates that no specific comment was provided for the respective entry. If there are multiple comments or missing entries ([nan]), it signifies that the respective argument was mentioned as many times as there are entries.</Data Structure>

<Task>Write two concise bullet points: one highlighting the main findings of the provided "rigid" and "soft" lists combined, and the other detailing the differences between the provided "rigid" and "soft" lists. Each set of bullet points should contain a maximum of five items, focusing on the overarching argument structures. Additionally, provide a summary paragraph of no more than four sentences that encapsulates the main findings and the found differences. Do not use the term list, instead refer to the {robots}. Be scientific and neutral in your wording. Consider all provided information carefully. Check if you have provided the two lists of bullet points (called mainFindings and differences), and the summary paragraph (called summary).</Task>"""


user_template = """List "rigid": 
{rigid}

List "soft": 
{soft}"""

# rescue robots and socially assistive robots
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

result = prompt_template.invoke({"robots": "rescue robots and socially assistive robots", "topicCategory": abbreviations_dict[sheet_name], "rigid": constant_comments_mapping, "soft": new_comments_mapping})
print(result)

print("result:", result)
print("result.to_messages():", result.to_messages())

messages=[SystemMessage(content='<Context>You are a researcher tasked with summarizing two wordlists that highlight people\'s assessments of rigid robots compared to soft robots. Laypersons were informed about the potential risks and benefits of rescue robots and socially assistive robots through scenario texts. Initially, they listed their perceived risks and benefits of rigid robots in a list titled "rigid." Subsequently, they learned about the trend towards soft robots, which are made of flexible, soft materials and are electronic-free. They then created a list titled "soft" to highlight the differences between rigid and soft robots. The overarching topic of the two lists is the perceived trust.</Context>\n\n<Data Structure>The lists "rigid" and "soft" are dictionaries where the keys are written arguments, and their corresponding values are one or more comments related to those arguments. The value [nan] indicates that no specific comment was provided for the respective entry. If th

## Provide schemas for structured outputs

In [14]:
json_schema = {
    "title": "Outputs",
    "description": "Bullet lists detailing the similarities and differences between the rigid and soft lists and a summary paragraph.",
    "type": "object",
    "properties": {
        "mainFindings": {
            "type": "string",
            "description": "Bullet lists highlighting the main findings of the provided rigid and soft lists",
        },
        "differences": {
            "type": "string",
            "description": "Bullet lists detailing the differences between the rigid and soft lists",
        },
          "summary": {
            "type": "string",
            "description": "Summary paragraph that provides a summary of the main findings and the found differences",
        },
    },
    "required": ["similarities", "differences", "summary"],
}

## Define model parameters

In [15]:
def basic_API_call(
    prompt,
    robots,
    topicCategory,
    openai_api_key,
    dictonaryRigid,
    dictonarySoft,
    json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
):

    # prompt = PromptTemplate(template=template)
    seed = 123

    model = ChatOpenAI(model=model_name, openai_api_key=openai_api_key, max_tokens=max_tokens, model_kwargs={"seed": seed}, temperature=0.0)
       
    structured_llm = model.with_structured_output(json_schema, include_raw=True)
    chain = prompt | structured_llm

    with get_openai_callback() as cb:
        response = chain.invoke(
            {"robots": robots, "topicCategory": topicCategory, "rigid": dictonaryRigid, "soft": dictonarySoft}
        )
        print(cb)
    
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
        
    return response

# Run ChatGPT

## Example (overall)

> Remark: The argument structures between the two types of robots differ significantly. Therefore, the robots are qualitatively summarized separately to ensure a clear and accurate comparison.

In [16]:
print(f"sheet_name: {sheet_name}")
print(f"abbreviations_dict[sheet_name]: {abbreviations_dict[sheet_name]}")


result = basic_API_call(prompt=prompt_template,
    robots="rescue robots and socially assistive robots",
    topicCategory=abbreviations_dict[sheet_name],
    openai_api_key=key.openai_api_key,
    dictonaryRigid=constant_comments_mapping, # overall
    dictonarySoft=new_comments_mapping,
    json_schema=json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
)

sheet_name: T
abbreviations_dict[sheet_name]: perceived trust
Tokens Used: 2332
	Prompt Tokens: 1965
	Completion Tokens: 367
Successful Requests: 1
Total Cost (USD): $0.01533
Total Tokens: 2332
Prompt Tokens: 1965
Completion Tokens: 367
Total Cost (USD): $0.01533


In [17]:
# Extract the 'parsed' section from the JSON data
parsed_section = result.get('parsed', {})
#print(parsed_section)
# Extract the translations
mainFindings = parsed_section.get('mainFindings')
differences = parsed_section.get('differences')
summary = parsed_section.get('summary')

print(f"result (raw): {result}")
print(f"mainFindings: {mainFindings}")
print(f"differences: {differences}")
print(f"summary: {summary}")

result (raw): {'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_su5QTyORCSQExEOtR6rDMC8f', 'function': {'arguments': '{"mainFindings":"- Rigid robots are perceived as reliable, available 24/7, and capable of performing tasks without emotional bias.\\n- Both rigid and soft robots are seen as capable of providing individualized care and adapting to specific needs.\\n- Rigid robots are noted for their ability to make fact-based, objective decisions and provide consistent performance.\\n- Soft robots are highlighted for their adaptability to rescue situations and potential to access areas inaccessible to humans.\\n- Both types of robots are considered less prone to errors compared to humans.","differences":"- Rigid robots are emphasized for their robustness, resilience, and ability to perform under severe conditions, while soft robots are noted for their flexibility and adaptability.\\n- Rigid robots are seen as neutral and unbiased, making decisions based on fac

## Separately for robots (rescue robot and socially assistive robot)

### for rescue robots

In [20]:
categories = []
mainFindings = []
differences = []
summary = []
rawResults = []


for category in abbreviations_dict.keys():
    print(f"category: {category}")
    
    # do not process the rest categories
    if category not in ['RCPP', 'RCPN', 'RCA', 'RCN']:
        df = all_sheets_RR[category]
        constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
        # print("mapping constant x comments:", constant_comments_mapping)
        # print(len(constant_comments_mapping))
        
        new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
        # print("mapping new x comments:", new_comments_mapping)
        # print(len(new_comments_mapping))
    
        result = basic_API_call(prompt=prompt_template,
            robots="rescue robots",
            topicCategory=abbreviations_dict[category],
            openai_api_key=key.openai_api_key,
            dictonaryRigid=constant_comments_mapping,
            dictonarySoft=new_comments_mapping,
            json_schema=json_schema,
            model_name="gpt-4o",
            max_tokens=1600, # increase limit
        )
        
        # append raw results
        categories.append(category)
        rawResults.append(result)
        
        # append parsed results
        parsed_section = result.get('parsed', {})
        mainFindings.append(parsed_section.get('mainFindings'))
        differences.append(parsed_section.get('differences'))
        summary.append(parsed_section.get('summary'))
        #print("length of mainFindings:", len(parsed_section.get('mainFindings')))
        #print("length of differences:", len(parsed_section.get('differences')))
        #print("length of summary:", len(parsed_section.get('summary')))
        
# save file
df_RR = pd.DataFrame({
    'Category': categories,
    'mainFindings': mainFindings,
    'differences': differences,
    'summary': summary,
    'rawResults' : rawResults
})

# Path to your Excel file
file_path = directory + "/output/" + "rescue robot_ChatGPT" + ".xlsx"
# save the dataframe to an Excel file
df_RR.to_excel(file_path, index=False)

category: RCPP
category: LC
Tokens Used: 905
	Prompt Tokens: 617
	Completion Tokens: 288
Successful Requests: 1
Total Cost (USD): $0.007404999999999999
Total Tokens: 905
Prompt Tokens: 617
Completion Tokens: 288
Total Cost (USD): $0.007404999999999999
length of mainFindings: 381
length of differences: 535
length of summary: 577
category: T
Tokens Used: 1407
	Prompt Tokens: 1088
	Completion Tokens: 319
Successful Requests: 1
Total Cost (USD): $0.010225000000000001
Total Tokens: 1407
Prompt Tokens: 1088
Completion Tokens: 319
Total Cost (USD): $0.010225000000000001
length of mainFindings: 502
length of differences: 646
length of summary: 595
category: SIP
Tokens Used: 1333
	Prompt Tokens: 1027
	Completion Tokens: 306
Successful Requests: 1
Total Cost (USD): $0.009725
Total Tokens: 1333
Prompt Tokens: 1027
Completion Tokens: 306
Total Cost (USD): $0.009725
length of mainFindings: 520
length of differences: 580
length of summary: 536
category: HRIP
Tokens Used: 899
	Prompt Tokens: 615
	Com

### for socially assistive robots

In [21]:
categories = []
mainFindings = []
differences = []
summary = []
rawResults = []


for category in abbreviations_dict.keys():
    print(f"category: {category}")
    
    # do not process the rest categories
    if category not in ['RCPP', 'RCPN', 'RCA', 'RCN']:
        df = all_sheets_SAR[category]
        constant_comments_mapping = create_multivalue_dict(df, 'constant', 'constant_comments')
        # print("mapping constant x comments:", constant_comments_mapping)
        # print(len(constant_comments_mapping))
        
        new_comments_mapping = create_multivalue_dict(df, 'new', 'new_comments')
        # print("mapping new x comments:", new_comments_mapping)
        # print(len(new_comments_mapping))
    
        result = basic_API_call(prompt=prompt_template,
            robots="socially assistive robots",
            topicCategory=abbreviations_dict[category],
            openai_api_key=key.openai_api_key,
            dictonaryRigid=constant_comments_mapping,
            dictonarySoft=new_comments_mapping,
            json_schema=json_schema,
            model_name="gpt-4o",
            max_tokens=1000,
        )
        
        # append raw results
        categories.append(category)
        rawResults.append(result)
        
        # append parsed results
        parsed_section = result.get('parsed', {})
        mainFindings.append(parsed_section.get('mainFindings'))
        differences.append(parsed_section.get('differences'))
        summary.append(parsed_section.get('summary'))
        
# save file
df_SAR = pd.DataFrame({
    'Category': categories,
    'mainFindings': mainFindings,
    'differences': differences,
    'summary': summary,
    'rawResults' : rawResults
})

# Path to your Excel file
file_path = directory + "/output/" + "socially assistive robot_ChatGPT" + ".xlsx"
# save the dataframe to an Excel file
df_SAR.to_excel(file_path, index=False)

category: RCPP
category: LC
Tokens Used: 1137
	Prompt Tokens: 840
	Completion Tokens: 297
Successful Requests: 1
Total Cost (USD): $0.008655
Total Tokens: 1137
Prompt Tokens: 840
Completion Tokens: 297
Total Cost (USD): $0.008655
category: T
Tokens Used: 1654
	Prompt Tokens: 1347
	Completion Tokens: 307
Successful Requests: 1
Total Cost (USD): $0.01134
Total Tokens: 1654
Prompt Tokens: 1347
Completion Tokens: 307
Total Cost (USD): $0.01134
category: SIP
Tokens Used: 2794
	Prompt Tokens: 2496
	Completion Tokens: 298
Successful Requests: 1
Total Cost (USD): $0.01695
Total Tokens: 2794
Prompt Tokens: 2496
Completion Tokens: 298
Total Cost (USD): $0.01695
category: HRIP
Tokens Used: 2197
	Prompt Tokens: 1910
	Completion Tokens: 287
Successful Requests: 1
Total Cost (USD): $0.013855
Total Tokens: 2197
Prompt Tokens: 1910
Completion Tokens: 287
Total Cost (USD): $0.013855
category: AN
Tokens Used: 1411
	Prompt Tokens: 1100
	Completion Tokens: 311
Successful Requests: 1
Total Cost (USD): $0.0